# Getting images from HiCAT simulator and controlling the IrisAO

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import hicat.simulators

## Instantiate hicat

In [ ]:
hc = hicat.simulators.hicat_sim.HICAT_Sim()

In [ ]:
hc.testbed_state

In [ ]:
print(hc.describe())

## Set the testbed into the correct hardware state

In [ ]:
hc.pupil_maskmask = 'circular'    # I will likely have to implement a new pupil mask
hc.iris_ao = 'iris_ao'
hc.apodizer = 'no_apodizer'
hc.lyot_stop = 'circular'
hc.detector = 'imager'

In [ ]:
hc.testbed_state

## Get an image from the simulator with flat IrisAO, unnormalized

The initial simulator just read the info from the configfile. Here, we're just trying to make a PSF and we will deal with the setup later.

In [ ]:
# All the images
plt.figure(figsize=(14,14))
psf, waves = hc.calc_psf(display=True, return_intermediates=True)
plt.show()

In [ ]:
hicat_psf = psf[0].data
print(type(hicat_psf))
print(hicat_psf.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.log10(hicat_psf))
plt.show()

## Get normalized coro PSF

### Calculate direct image

In [ ]:
hc.inculde_fpm = False
psf_direct_data = hc.calc_psf()

In [ ]:
psf_direct = psf_direct_data[0].data
norm = psf_direct.max()

plt.figure(figsize=(10,10))
plt.title('Direct PSF')
plt.imshow(np.log10(psf_direct), cmap='inferno')
plt.colorbar()

### Calculate normalized coro image

In [ ]:
hc.inculde_fpm = True
psf_coro_data = hc.calc_psf(display=False, return_intermediates=False)

In [ ]:
psf_coro = psf_coro_data[0].data/norm

plt.figure(figsize=(10,10))
plt.title('Coro PSF')
plt.imshow(np.log10(psf_coro), cmap='inferno')
plt.colorbar()